In [11]:
from datetime import datetime, timedelta
import random
import pandas as pd

# Generate entry and exit timestamps for each MAC address
entries = []
num_records = 2000

df = pd.read_csv("../mac_roll_data/mac.csv")


mac_list = df['mac'].tolist()





for _ in range(num_records):
    mac = random.choice(mac_list)
    # Random date within the past month
    base_date = datetime.now() - timedelta(days=random.randint(1, 30))
    # Random entry time during the day
    entry_time = base_date.replace(hour=random.randint(7, 20), minute=random.randint(0, 59), second=random.randint(0, 59))
    # Random exit time within 1 to 4 hours after entry
    exit_time = entry_time + timedelta(hours=random.randint(0, 4), minutes=random.randint(0, 59))
    
    entries.append([mac, entry_time.strftime("%Y-%m-%d %H:%M:%S"), exit_time.strftime("%Y-%m-%d %H:%M:%S")])

# Create DataFrame
df = pd.DataFrame(entries, columns=['mac', 'exit_datetime','entry_datetime'])
df.head()


,mac,exit_datetime,entry_datetime
0,39:8e:20:c1:f1:cc,2025-04-09 13:05:49,2025-04-09 17:34:49
1,dc:0c:58:c8:f1:97,2025-04-10 08:49:56,2025-04-10 11:39:56
2,12:d9:ad:19:f8:5d,2025-03-21 19:03:04,2025-03-21 21:35:04
3,28:b9:61:91:ae:ec,2025-03-17 18:19:59,2025-03-17 21:31:59
4,be:4e:6a:4e:83:ed,2025-04-09 14:52:16,2025-04-09 18:55:16


In [12]:
df.head()

,mac,exit_datetime,entry_datetime
0,39:8e:20:c1:f1:cc,2025-04-09 13:05:49,2025-04-09 17:34:49
1,dc:0c:58:c8:f1:97,2025-04-10 08:49:56,2025-04-10 11:39:56
2,12:d9:ad:19:f8:5d,2025-03-21 19:03:04,2025-03-21 21:35:04
3,28:b9:61:91:ae:ec,2025-03-17 18:19:59,2025-03-17 21:31:59
4,be:4e:6a:4e:83:ed,2025-04-09 14:52:16,2025-04-09 18:55:16


In [13]:
df.to_csv("entry_exit.csv", index=False)

## Upload to DB

In [14]:
# Requirements
host_name = "192.168.137.1"
user_name = "root"
user_password = "12341234"
database_name = "trial3" #overall database name
table_name = "exit_entry_logs" #table name



In [15]:
import pandas as pd
import mysql.connector


# Define your database connection details
conn = mysql.connector.connect(
    host=host_name,
        user=user_name,
        password=user_password,
        database=database_name)
cursor = conn.cursor()



# Prepare insert query dynamically
cols = ", ".join(df.columns)
placeholders = ", ".join(["%s"] * len(df.columns))
insert_query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"

# Convert DataFrame rows to list of tuples and insert
data = [tuple(x) for x in df.to_numpy()]

# Execute in bulk for performance
cursor.executemany(insert_query, data)

# Commit changes
conn.commit()
cursor.close()
conn.close()

print(f"Successfully inserted {len(data)} rows into '{table_name}'!")


Successfully inserted 2000 rows into 'exit_entry_logs'!
